<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Stanford_Dataset_to_YOLO_Format_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import os
import cv2

In [36]:
dataset_size = 6000

In [2]:
dataset_path = "/content/drive/MyDrive/data-yolov5"

In [ ]:
'''
for root, dirs, files in os.walk(dataset_path):
    for filename in files:
        print(filename)
'''

In [32]:
video_count = 0

For every video in videos, open the corresponding annotations file

In [33]:
for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    #print(str(dirpath) + " , " + str(dnames) + " , " + str(fnames))
    for f in fnames:
        if f == 'video.mov':
          video_count += 1

In [ ]:
print("Number of videos is: " + str(video_count))

**dataset_size/video_count** is the number of frames we will extract from each video

In [37]:
num_frames = int(dataset_size/video_count)

In [ ]:
for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    #print(str(dirpath) + " , " + str(dnames) + " , " + str(fnames))
    for f in fnames:
        if f == 'video.mov':
          video_path = dirpath + "/video.mov"
          annot_path = dirpath.replace("videos", "annotations") + "/annotations.txt"
          #print(annot_path)
          #print(video_path)

          ##### Process the video #####
          vidcap = cv2.VideoCapture(video_path)
          success, image = vidcap.read()

          frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

          count = 0

          while success:
            if count % num_frames == 0:
              #print(count)
              write_to = ""

              '''
              if count % ratio == 0:
                write_to = path_val + "/frame" + str(count) + ".jpg"
              else:
                write_to = path_train + "/frame" + str(count) + ".jpg"
              
              cv2.imwrite(write_to, image)
              #cv2.imwrite(path_images + "/frame%d.jpg" % count, image)     # save frame as JPEG file
              '''
              '''
              #print('Read a new frame: ', success)
              '''
            success, image = vidcap.read()
            count += 1
          
          #############################
